In [8]:
import os
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
! kaggle datasets download ninadaithal/imagesoasis

Dataset URL: https://www.kaggle.com/datasets/ninadaithal/imagesoasis
License(s): apache-2.0


In [10]:
! unzip imagesoasis.zip

Streaming output truncated to the last 5000 lines.
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_102.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_103.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_104.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_105.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_106.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_107.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_108.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_109.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_110.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_111.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_112.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_113.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_114.jpg  
  inflating: Data/Very mild Dementia/OAS1_0247_MR1_mpr-4_115.jpg  
  inflating

In [11]:
import os
import shutil
import random

# Define paths
original_dataset_dir = "/content/Data"
new_dataset_dir = "/content/New_Data"

# Ensure new dataset directory exists
os.makedirs(new_dataset_dir, exist_ok=True)

# Define number of samples per class (35k total, distributed across classes)
target_samples = 35000
num_classes = 4  # Update if needed
samples_per_class = target_samples // num_classes

# Sample files from each class
for class_name in os.listdir(original_dataset_dir):
    class_path = os.path.join(original_dataset_dir, class_name)
    new_class_path = os.path.join(new_dataset_dir, class_name)

    if os.path.isdir(class_path):
        os.makedirs(new_class_path, exist_ok=True)

        # Get all images in the class
        all_images = os.listdir(class_path)
        selected_images = random.sample(all_images, min(samples_per_class, len(all_images)))

        # Copy selected images
        for img in selected_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(new_class_path, img))

print("Subset dataset created with 35k images.")

Subset dataset created with 35k images.


In [12]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import  EfficientNetB0
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
# Set paths
DATASET_PATH = "/content/New_Data"
IMAGE_SIZE = (48, 48)
BATCH_SIZE = 32
TEST_SPLIT = 0.1
VAL_SPLIT = 0.1

# Class names
CLASSES = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

# Created a directory for organized dataset (Train, Val, Test)
OUTPUT_DIR = "processed_dataset"
os.makedirs(OUTPUT_DIR, exist_ok=True)

for split in ["train", "val", "test"]:
    for cls in CLASSES:
        os.makedirs(os.path.join(OUTPUT_DIR, split, cls), exist_ok=True)

In [14]:
# Function to split dataset into train/val/test
def split_and_copy_images():
    for cls in CLASSES:
        cls_path = os.path.join(DATASET_PATH, cls)
        images = os.listdir(cls_path)

        # Shuffle images
        np.random.shuffle(images)

        # Train-Val-Test Split
        train_imgs, temp_imgs = train_test_split(images, test_size=(VAL_SPLIT + TEST_SPLIT))
        val_imgs, test_imgs = train_test_split(temp_imgs, test_size=(TEST_SPLIT / (VAL_SPLIT + TEST_SPLIT)))

        # Copy images to new directory
        for img in train_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(OUTPUT_DIR, "train", cls, img))
        for img in val_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(OUTPUT_DIR, "val", cls, img))
        for img in test_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(OUTPUT_DIR, "test", cls, img))

In [15]:
from sklearn.model_selection import train_test_split
split_and_copy_images()

# Data augmentation for training set to handle class imbalance
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)

# No augmentation for validation and test sets
test_val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

In [16]:
# Load data using ImageDataGenerator
train_generator = train_datagen.flow_from_directory(
    os.path.join(OUTPUT_DIR, "train"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = test_val_datagen.flow_from_directory(
    os.path.join(OUTPUT_DIR, "val"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_generator = test_val_datagen.flow_from_directory(
    os.path.join(OUTPUT_DIR, "test"),
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)
print("Class indices:", train_generator.class_indices)

Found 18391 images belonging to 4 classes.
Found 2299 images belonging to 4 classes.
Found 2300 images belonging to 4 classes.
Class indices: {'Mild Dementia': 0, 'Moderate Dementia': 1, 'Non Demented': 2, 'Very mild Dementia': 3}


In [17]:
# Compute class weights
from sklearn.utils.class_weight import compute_class_weight
labels = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))

In [20]:
# Load EfficientnetB0 (pretrained on ImageNet)
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(48, 48, 3))

# Freeze base model layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 2, 2, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │           1,028 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,378,535 (16.70 MB)

 Trainable params: 328,964 (1.25 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [21]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    class_weight=class_weights
)

Epoch 1/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 80s 102ms/step - accuracy: 0.2254 - loss: 1.5198 - val_accuracy: 0.2175 - val_loss: 1.3678
Epoch 2/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 54s 70ms/step - accuracy: 0.1363 - loss: 1.4272 - val_accuracy: 0.0213 - val_loss: 1.3936
Epoch 3/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 40s 69ms/step - accuracy: 0.0913 - loss: 1.3674 - val_accuracy: 0.2175 - val_loss: 1.3859
Epoch 4/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 39s 68ms/step - accuracy: 0.0588 - loss: 1.3815 - val_accuracy: 0.0213 - val_loss: 1.3910
Epoch 5/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 39s 68ms/step - accuracy: 0.0453 - loss: 1.3875 - val_accuracy: 0.0213 - val_loss: 1.3904
Epoch 6/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 40s 69ms/step - accuracy: 0.0299 - loss: 1.4059 - val_accuracy: 0.3806 - val_loss: 1.3861
Epoch 7/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 39s 68ms/step - accuracy: 0.1037 - loss: 1.4161 - val_accuracy: 0.2175 - val_loss: 1.3847
Epoch 8/15
575/575 ━━━━━━━━━━━━━━━━━━━━ 38s 66ms/step - accuracy: 0.2361 - loss: 1.3976 -

In [22]:
# Save model
model.save("efficientnet_oasis_15_epochs.keras")

In [23]:
# Get train,val and test accuracies
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(val_generator)
test_loss, test_acc = model.evaluate(test_generator)

print(f"Train Accuracy: {train_acc * 100:.2f}%")
print(f"Val Accuracy: {val_acc * 100:.2f}%")
print(f"Test Accuracy: {test_acc * 100:.2f}%")

575/575 ━━━━━━━━━━━━━━━━━━━━ 41s 71ms/step - accuracy: 0.3804 - loss: 1.3857
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.4004 - loss: 1.3856
72/72 ━━━━━━━━━━━━━━━━━━━━ 7s 103ms/step - accuracy: 0.0910 - loss: 1.3859
Train Accuracy: 38.06%
Val Accuracy: 38.06%
Test Accuracy: 38.04%


In [24]:
from sklearn.metrics import classification_report

# Assuming you have the true labels and predicted labels for both train and test datasets
val_predictions = model.predict(val_generator)
test_predictions = model.predict(test_generator)

# Convert predictions to class labels (if necessary, depends on your model output)
val_predictions_labels = np.argmax(val_predictions, axis=1)
test_predictions_labels = np.argmax(test_predictions, axis=1)

# Assuming your labels are categorical and represented as integers
print("Train Classification Report:")
print(classification_report(val_generator.classes, val_predictions_labels))

print("Test Classification Report:")
print(classification_report(test_generator.classes, test_predictions_labels))

# You can also extract individual metrics:
# For train
val_report = classification_report(val_generator.classes, val_predictions_labels, output_dict=True)
print(f"Val Precision: {val_report['weighted avg']['precision']:.2f}")
print(f"Val Recall: {val_report['weighted avg']['recall']:.2f}")
print(f"Val F1-Score: {val_report['weighted avg']['f1-score']:.2f}")

# For test
test_report = classification_report(test_generator.classes, test_predictions_labels, output_dict=True)
print(f"Test Precision: {test_report['weighted avg']['precision']:.2f}")
print(f"Test Recall: {test_report['weighted avg']['recall']:.2f}")
print(f"Test F1-Score: {test_report['weighted avg']['f1-score']:.2f}")

72/72 ━━━━━━━━━━━━━━━━━━━━ 18s 126ms/step
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 89ms/step
Train Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       500
           1       0.00      0.00      0.00        49
           2       0.00      0.00      0.00       875
           3       0.38      1.00      0.55       875

    accuracy                           0.38      2299
   macro avg       0.10      0.25      0.14      2299
weighted avg       0.14      0.38      0.21      2299

Test Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       501
           1       0.00      0.00      0.00        49
           2       0.00      0.00      0.00       875
           3       0.38      1.00      0.55       875

    accuracy                           0.38      2300
   macro avg       0.10      0.25      0.14      2300
weighted avg       0.14      0.38      0.21   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m